In [36]:
import json
with open("/home/sora/llm/moe/output/deepseek_prunese_again_11/raw_result_boolq.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print(len(data))

3270


In [ ]:
total = 0
correct = 0
for sample in data:
    if sample['model_generation'] is not None :
        total += 1
        gold = str(sample['gold_answer']).strip().lower()
        pred = sample["model_generation"].strip().lower()
        if gold == pred:
            correct+=1
correct/ total

In [37]:
import re
total = 0
correct = 0
pattern = r"\b(?:Yes|No)\b"
for sample in data:
    if re.search(pattern, sample['answer']):
        total += 1
        gold = str(sample['gold']).strip().lower()
        pred = "false" if re.search(r"\bNo\b", sample["answer"]) else "true"
        if gold == pred:
            correct+=1
correct/ total

0.629352137505509

In [19]:
import json
import csv
import sys

def jsonl_to_csv(jsonl_file_path, csv_file_path):
    """
    Converts a JSONL file to a CSV file.

    Args:
        jsonl_file_path (str): The path to the input JSONL file.
        csv_file_path (str): The path to the output CSV file.
    """
    try:
        # Step 1: Read the JSONL file and collect all objects
        data = []
        with open(jsonl_file_path, 'r', encoding='utf-8') as f_jsonl:
            for line in f_jsonl:
                # Skip empty lines to avoid errors
                if line.strip():
                    data.append(json.loads(line))

        if not data:
            print("The JSONL file is empty or contains no valid JSON objects.")
            return

        # Step 2: Extract column headers from the keys of the first JSON object
        # We assume all objects have the same keys for a clean CSV
        headers = list(data[0].keys())

        # Step 3: Write the data to the CSV file
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as f_csv:
            writer = csv.DictWriter(f_csv, fieldnames=headers)
            
            # Write the header row
            writer.writeheader()
            
            # Write the data rows
            writer.writerows(data)
            
        print(f"Successfully converted '{jsonl_file_path}' to '{csv_file_path}'. 🎉")

    except FileNotFoundError:
        print(f"Error: The file '{jsonl_file_path}' was not found.")
        sys.exit(1)
    except json.JSONDecodeError as e:
        print(f"Error: Could not decode JSON from the file. Check for malformed JSON objects.")
        print(f"Details: {e}")
        sys.exit(1)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        sys.exit(1)

# Example usage:
if __name__ == "__main__":

    # Define the file paths
    input_file = '/home/sora/llm/moe/output/deepseek_pre/final_result.jsonl'
    output_file = '/home/sora/llm/moe/output/deepseek_pre/final_result.csv'
    
    # Run the conversion
    jsonl_to_csv(input_file, output_file)

Successfully converted '/home/sora/llm/moe/output/deepseek_pre/final_result.jsonl' to '/home/sora/llm/moe/output/deepseek_pre/final_result.csv'. 🎉


In [20]:
import json
import string
import sys
from typing import List, Dict

def add_full_choice(data: List[Dict]) -> List[Dict]:
    """Add a 'full_choice' field like 'D. 0,4' to each sample."""
    letters = list(string.ascii_uppercase)
    
    for sample in data:
        gold = sample.get("gold")
        choices = sample.get("choices", [])
        
        if not gold or gold not in letters:
            sample["full_choice"] = None
            continue
        
        idx = letters.index(gold)
        if idx < len(choices):
            sample["full_choice"] = f"{gold}. {choices[idx]}"
        else:
            sample["full_choice"] = None
    return data

if __name__ == "__main__":

    input_path = "/home/sora/llm/moe/output/deepseek_pre/raw_result_baseline_arc.json"
    output_path = "/home/sora/llm/moe/output/deepseek_pre/raw_result_baseline_arc.json"

    # Load JSON file
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # If the file contains a single object, wrap it in a list
    if isinstance(data, dict):
        data = [data]
    if isinstance(data, str):
        data = data.split("\n")
    # Process
    updated_data = add_full_choice(data)

    # Save back
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(updated_data if len(updated_data) > 1 else updated_data[0], f, indent=2, ensure_ascii=False)

    print(f"Updated file saved to {output_path}")


Updated file saved to /home/sora/llm/moe/output/deepseek_pre/raw_result_baseline_arc.json


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Use light style
plt.style.use("default")   # or "seaborn-v0_8-whitegrid" for gridlines

# Data from the table
data = {
    "Baseline": [39.10, 41.46, 18.95, 67.74, 36.08, 31.91],
    "Pruning (SE)": [30.99, 30.72, 0.03, 62.75, 26.94, 31.02],
    "Pruning (non SE)" : [40.71, 39.51, 10.61, 68.19, 36.99, 31.23],
    "Finetune SE only": [44.98, 52.17, 15.77, 69.87, 39.98, 33.6],
    "Finetune non-SE only": [45.23, 52.26, 15.61, 69.87, 39.98, 33.64],
    "Finetune QKV": [45.16, 53.71, 15.54, 69.87, 39.90, 31.91],
}

benchmarks = ["MMLU", "ARC-E", "GSM8K", "BoolQ", "MedicalQA", "VMLU"]

# Put into DataFrame
df = pd.DataFrame(data, index=benchmarks)

# Plot grouped bar chart
df.plot(kind="bar", figsize=(12, 6), edgecolor="black")

plt.title("Performance Comparison by Benchmark")
plt.ylabel("Score")
plt.xticks(rotation=30, ha="right")
plt.legend(title="Method", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


In [21]:
import json

filename = "/home/sora/llm/moe/output/deepseek_pre/raw_result_med.json"

# Read original JSON
with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Process each sample
for sample in data:
    choices_dict = {}
    for line in sample["choices"].strip().split("\n"):
        if ". " in line:
            key, _ = line.split(". ", 1)
            choices_dict[key.strip()] = line.strip()

    gold_key = sample.get("gold", "")
    sample["full_choice"] = choices_dict.get(gold_key, "")

# Write back to the same file
with open(filename, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)


In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained("/home/sora/llm/moe/ckpt/DeepSeek-V2-Lite-Chat-Pruned", trust_remote_code=True, device_map="cuda:1")
import json
import torch

# assume your model is already loaded as `model`
with open("/home/sora/llm/moe/output_experts/deepseek2/super_experts_info/super_experts_report/Super Experts Report.json") as f:
    expert_data = json.load(f)

for entry in expert_data:
    layer_idx = int(entry["layer_index"])
    expert_idx = int(entry["expert_index"])

    # navigate: model.layers[layer_idx].mlp.experts[expert_idx]
    expert = model.layers[layer_idx].mlp.experts[expert_idx]

    print(f"Layer {layer_idx}, Expert {expert_idx}")
    print(" gate_proj weights:", expert.gate_proj.weight.data)
    print(" up_proj weights:", expert.up_proj.weight.data)
    print(" down_proj weights:", expert.down_proj.weight.data)
    print("-" * 50)


In [ ]:
choices = [extract_and_convert(raw['raw']) for raw in data]

In [ ]:
gold = [sample['gold'] for sample in data]

In [ ]:
acc = 0
for i in range(len(choices)):
    if choices[i] == gold[i]:
        acc += 1
acc

In [ ]:
from datasets import load_dataset
dataset = load_dataset("Nikity/Kyoto-Corpus", split="train")

In [ ]:
dataset[0]